In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.transforms as tfs
from scipy.signal import wiener
import collections
from scipy import stats
from matplotlib.ticker import MaxNLocator
import json  
import os
import seaborn as sns
import scikit_posthocs as sp
from IPython.display import clear_output
import networkx as nx
from bct.utils import BCTParamError, normalize, get_rng

import warnings
import matplotlib
from scipy.spatial.distance import pdist,squareform
from scipy.cluster.hierarchy import linkage,dendrogram,fcluster
from statsmodels.sandbox.stats.runs import runstest_1samp 
import math
warnings.filterwarnings('ignore')

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from IPython.core.display import display, HTML
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius, RadiusCorridor
from pyrqa.metric import EuclideanMetric, MaximumMetric, TaxicabMetric
from pyrqa.computation import RQAComputation
from pyrqa.computation import RPComputation
from pyrqa.image_generator import ImageGenerator
import umap
from pyrqa.neighbourhood import Unthresholded
plt.rcParams['svg.fonttype'] = 'none'

display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
def plot_rate(raster,ci,c,bin_=4,fps=4,r=1.5):
    N,F=raster.shape
    indices_ci=np.where(ci==c)[0]
    raster_cluster=raster[indices_ci,:]
    y=np.sum(raster_cluster,axis=0)
    rate=np.zeros(y.shape)
    bin_b=0
    for j in range(y.shape[0]):
        rate[j]=np.sum(y[j-bin_b:j+1])
        if bin_b<bin_:
            bin_b+=1
    fpm=fps*60
    x=np.arange(0,F)/fpm
    fig=plt.figure(figsize=(12,3))
    #ax=plt.axes((0.05,0.35,0.75,0.9))
    plt.plot(x,rate,color='black')
    plt.xlim(np.min(x),np.max(x))
    plt.ylim(0,np.max(rate)+1)
    rate[np.where(rate==0)[0]]=np.nan
    time_series = TimeSeries(rate,embedding_dimension=2,time_delay=1)
    settings = Settings(time_series,
                    analysis_type=Classic,
                    neighbourhood=FixedRadius(r),    
                    #neighbourhood=Unthresholded(),
                    similarity_measure=EuclideanMetric,
                    theiler_corrector=1)
    computation = RPComputation.create(settings)
    result = computation.run()
    return result.recurrence_matrix

In [ ]:
def get_recurrence_analysis(raster,ci,r=1.5,bin_=4):
    df=pd.DataFrame(columns=['Ensemble','N','RR','DET','L','L_max','DIV','L_entr','LAM','TT','V_max','V_entr','W','W_max','W_div','W_entr'])
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        y=np.sum(raster_cluster,axis=0)
        rate=np.zeros(y.shape)
        bin_b=0
        for j in range(y.shape[0]):
            rate[j]=np.sum(y[j-bin_b:j+1])
            if bin_b<bin_:
                bin_b+=1
        rate[np.where(rate==0)[0]]=np.nan
        time_series = TimeSeries(rate,embedding_dimension=2,time_delay=1)

        settings = Settings(time_series,
                            analysis_type=Classic,
                            neighbourhood=FixedRadius(r),
                            similarity_measure=EuclideanMetric,
                            theiler_corrector=1)
        computation = RQAComputation.create(settings,
                                        verbose=True)

        result = computation.run()
        result.min_diagonal_line_length = 2
        result.min_vertical_line_length = 2
        result.min_white_vertical_line_length = 2
        fila={'Ensemble':i,'N':len(indices_ci),
              'RR':result.recurrence_rate,'DET':result.determinism,
              'L':result.average_diagonal_line,'L_max':result.longest_diagonal_line,'DIV':result.divergence,
              'L_entr':result.entropy_diagonal_lines,'LAM':result.laminarity,
              'TT':result.laminarity,'V_max':result.longest_vertical_line,
              'V_entr':result.entropy_vertical_lines,'W':result.average_white_vertical_line,
              'W_max':result.longest_white_vertical_line,'W_div':result.longest_white_vertical_line_inverse,
              'W_entr':result.entropy_white_vertical_lines}
        df=df.append(fila, ignore_index=True)
        clear_output()
    v_max=1
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        y=np.sum(raster_cluster,axis=0)
        rate=np.zeros(y.shape)
        bin_b=0
        for j in range(y.shape[0]):
            rate[j]=np.sum(y[j-bin_b:j+1])
            if bin_b<bin_:
                bin_b+=1
        rate[np.where(rate==0)[0]]=np.nan
        time_series = TimeSeries(rate,embedding_dimension=2,time_delay=1)
        settings = Settings(time_series,
                        analysis_type=Classic,
                        neighbourhood=FixedRadius(r),    
                        #neighbourhood=Unthresholded(),
                        similarity_measure=EuclideanMetric,
                        theiler_corrector=1)
        computation = RPComputation.create(settings)
        result = computation.run()
        plt.matshow(result.recurrence_matrix,cmap='Blues',vmin=0, vmax=v_max)
        plt.colorbar()
    
    return df

In [ ]:
def get_transition_graph(transition_matrix,cmap):
    G = nx.DiGraph()
    color_map=[]
    for i in range(len(transition_matrix)):
        G.add_node(i)
        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
            color_map.append(cmap(i/(len(transition_matrix)-1)))
        else:
            color_map.append(cmap(i))
    for i in range(len(transition_matrix)):
        for j in range(len(transition_matrix)):
            if transition_matrix[i,j]>=1:
                if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                    G.add_edge(i,j, weight=transition_matrix[i,j],color=cmap(i/(len(transition_matrix)-1))) 
                else:
                    G.add_edge(i,j, weight=transition_matrix[i,j],color=cmap(i))    
    return G,color_map

In [ ]:
def get_transition_matrix(colores,ci):
    secuencia=[-1]
    for key,values in colores.items():
        if len(values)>=1:
            for value in values:
                if secuencia[-1]!=value:
                    secuencia.append(value)
                    break
    del secuencia[0]
    transition_matrix=np.zeros((max(secuencia)+1,max(secuencia)+1))
    for i in range(len(secuencia)-1):
        transition_matrix[secuencia[i],secuencia[i+1]]+=1
    for i in range(max(ci)+1):
        #transition_matrix[i,:]=(transition_matrix[i,:]/np.sum(transition_matrix[i,:]))*100
        transition_matrix[i,:]=(transition_matrix[i,:]/np.sum(transition_matrix))*100
    return transition_matrix

In [ ]:
def get_3D_projection_vectors(raster,ci,n_cmap,window_percentage,n_sigma,min_dist,random_state,metric):
    #%matplotlib notebook
    cmap = matplotlib.cm.get_cmap(n_cmap)  
    colores={}
    for i in range(raster.shape[1]):
        colores[i]=[]
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]    
        peaks,th=get_tam_pico_sig(raster_cluster,window_percentage,n_sigma)
        for j,pk in enumerate(peaks):
            if pk:
                colores[j].append(i)
    raster_ensemble=np.empty((raster.shape[0],0), int)
    color_to_plot=[]
    count=0
    for i in range(raster.shape[1]):
        list_=colores[i]
        if len(list_)>0:
            for item in list_:
                raster_ensemble=np.append(raster_ensemble, np.expand_dims(raster[:,i], axis=1), axis=1)
                raster_ensemble[np.where(ci!=item)[0],count]=0
                color_to_plot.append(item)
                count+=1            
    
    mapper = umap.UMAP(min_dist=min_dist,n_components=3,random_state=random_state,metric=metric).fit(raster_ensemble.T)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for i in range(raster_ensemble.shape[1]):
        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
            ax.scatter(mapper.embedding_[i, 0], mapper.embedding_[i, 1],mapper.embedding_[i, 2],
                       s=15, color=cmap(color_to_plot[i]/max(color_to_plot)))
        else:
            ax.scatter(mapper.embedding_[i, 0], mapper.embedding_[i, 1],mapper.embedding_[i, 2],
                       s=15, color=cmap(color_to_plot[i]))
    
    return mapper,color_to_plot,colores

In [ ]:
def plot_ensamble_raster(raster,fps,cluster_index,name_colormap,n_sigma=2,window_percentage=20,markersize=5):
    cmap = matplotlib.cm.get_cmap(name_colormap)
    N,F=raster.shape
    actividad=np.zeros((N,1))
    orden_plot=[]
    
    plt.figure(figsize=(12,6))

    ax=plt.axes((0.05,0.35,0.75,0.6))
    count=0
    ensembles_in_time={}
    for i in range(raster.shape[1]):
        ensembles_in_time[i]=[]
    
    for ci in range(max(cluster_index)+1):
        indices_ci=np.where(cluster_index==ci)[0]
        coactividad_cluster=np.sum(raster[np.where(cluster_index==ci)[0],:],axis=0)
        peaks,th=get_tam_pico_sig(raster[np.where(cluster_index==ci)[0],:],window_percentage,n_sigma)
        
        idx=np.where(peaks==True)[0]
        for i in indices_ci:
            orden_plot.append(i)
            indices=np.where(raster[i,:]==1)[0]
            
            if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                plt.plot(indices,raster[i,indices]*(count+1),
                        marker='|',linestyle='None',
                        markersize=markersize,color=cmap(ci/max(cluster_index)),alpha=0.1)
                plt.plot(idx,raster[i,idx]*(count+1),
                        marker='s',linestyle='None',
                        markersize=markersize,color=cmap(ci/max(cluster_index)),alpha=1)
            else:
                plt.plot(indices,raster[i,indices]*(count+1),
                        marker='|',linestyle='None',
                        markersize=markersize,color=cmap(ci),alpha=0.1)
                plt.plot(idx,raster[i,idx]*(count+1),
                        marker='s',linestyle='None',
                        markersize=markersize,color=cmap(ci),alpha=1)
            actividad[count]=np.sum(raster[i,:])*100/F
            count+=1
        
        for j,pk in enumerate(peaks):
            if pk:
                ensembles_in_time[j].append(ci)       
        
    ax.set_xlim(0,F-1)
    ax.set_ylim(1,N)
    plt.xticks([])
    plt.ylabel("Neuron Label")

    ax=plt.axes(((0.05,0.12,0.75,0.2)))
    coactividad=np.sum(raster,axis=0)
    fpm=fps*60
    tiempo=np.arange(0,F)/fpm
    plt.plot(tiempo,coactividad,linewidth=0.5,color='black',alpha=1)
    ax.set_xlim(np.min(tiempo),np.max(tiempo))
    ax.set_ylim(0,ymax=np.max(coactividad)+1)
    plt.xlabel("Time (min)")
    plt.ylabel("Coactivity")
    ymax=np.max(coactividad)+1
    inicio=-1
    final=-1
    bin_=markersize+2
    for ci in range(max(cluster_index)+1):
        for key,value in ensembles_in_time.items():
            encontre=False
            if len(value)>0:
                for ensemble in value:
                    if ensemble==ci:
                        encontre=True
                        if inicio<0:
                            inicio=key
                            final=key
                        else:
                            final=key
                        break
                if encontre==False:
                    if final>0:
                        if inicio-bin_<0:
                            if final+bin_>F:
                                x=tiempo[0:F]
                            else:
                                x=tiempo[0:final+bin_]
                        elif final+bin_>F:
                            x=tiempo[inicio-bin_:F]
                        else:
                            x=tiempo[inicio-bin_:final+bin_]
                        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                            plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci/max(cluster_index)),alpha=0.5)
                        else:
                            plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci),alpha=0.5)
                        inicio=-1
                        final=-1
            if encontre==False:
                if final>0:
                    if inicio-bin_<0:
                        if final+bin_>F:
                            x=tiempo[0:F]
                        else:
                            x=tiempo[0:final+bin_]
                    elif final+bin_>F:
                        x=tiempo[inicio-bin_:F]
                    else:
                        x=tiempo[inicio-bin_:final+bin_]
                    if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                        plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci/max(cluster_index)),alpha=0.5)
                    else:
                        plt.fill_between(x, np.ones(x.shape)*ymax, np.zeros(x.shape),facecolor=cmap(ci),alpha=0.5)
                    inicio=-1
                    final=-1
                    
                    
    
    ax=plt.axes((0.85,0.35,0.1,0.6))
    plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
    ax.set_xlim(0,max(actividad)+1)
    ax.set_ylim(1,N)
    plt.xlabel("# Frames")
    plt.yticks([])
    return orden_plot,ensembles_in_time

In [ ]:
def test_coactivity_is_random(raster,ci,n_iter):
    for i in range(max(ci)+1):
        indices_ci=np.where(ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        print('Cluster',i)
        coactivity_is_random(raster_cluster,n_iter)
        print()

In [ ]:
def get_tam_pico_sig(raster,window_percentage=20,n_sigma=2):
    column = np.sum(raster,axis=0)
    N = len(column)
    time = np.arange(0,N)
    k = int(len(column) * (window_percentage/100))
    get_bands = lambda column : (np.mean(column) + n_sigma*np.std(column),np.mean(column) - n_sigma*np.std(column))
    bands = [get_bands(column[range(0 if i - k < 0 else i-k ,i + k if i + k < N else N)]) for i in range(0,N)]
    upper, lower = zip(*bands)
    anomalies = (column > upper) | (column < lower)
    
    for j in range(N):
        if anomalies[j]==True:
            if column[j]<2:
                anomalies[j]=False
    
    return anomalies,upper

In [ ]:
def plot_coactivity_ensambles(raster,cluster_index,n_iter,name_colormap,fps,window_percentage=20,n_sigma=2):
    Ci=cluster_index
    cmap = matplotlib.cm.get_cmap(name_colormap)
    N,F=raster.shape
    nmodules=max(Ci)+1
    maxCo=0

    for i in range(int(nmodules)):
        tempCo=np.sum(raster[np.where(Ci==i)[0],:],axis=0)
        if max(tempCo)>maxCo:
            maxCo=max(tempCo)

    fpm=fps*60
    x=np.arange(0,F)/fpm

    Co=np.zeros((int(nmodules),len(tempCo)))

    fig=plt.figure(figsize=(12,6))
    #ax=plt.axes((0.05,0.35,0.75,0.9))
    plt.subplots_adjust(hspace=0.000)
    number_of_subplots=nmodules
    ejes=[]
    for i in range(int(nmodules)):
        indices_ci=np.where(Ci==i)[0]
        raster_cluster=raster[indices_ci,:]
        peaks,th=get_tam_pico_sig(raster_cluster,window_percentage,n_sigma)     
        Co[i,:]=np.sum(raster[np.where(Ci==i)[0],:],axis=0)
        ax = plt.subplot(number_of_subplots,1,number_of_subplots-i)
        
        if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
            ax.plot(x,Co[i,:],color=cmap(i/max(Ci)),alpha=1)
            ax.plot(x,th,color=cmap(i/max(Ci)),alpha=0.2)
            plt.fill_between(x, th, np.zeros(x.shape),facecolor=cmap(i/max(Ci)),alpha=0.1)
        else:        
            ax.plot(x,Co[i,:],color=cmap(i),alpha=1)
            ax.plot(x,th,color=cmap(i),alpha=0.2)
            plt.fill_between(x, th, np.zeros(x.shape),facecolor=cmap(i),alpha=0.1)

        ax.set_xlim(np.min(x),np.max(x))
        ax.set_ylim(0,maxCo+1)
        if i==0:
            plt.xlabel("Time (min)")
            plt.ylabel("Coactivity")
        else:
            ax.set_xticks([])
        ejes.append(ax)

    for ax in ejes:
        pos=ax.get_position()
        pos=pos.from_extents(0.05,pos.y0,0.8,pos.y1)
        ax.set_position(pos)

In [ ]:
def coactivity_is_random(raster,n_iter):
    coactividad=np.sum(raster,axis=0)
    rt,p=runstest_1samp(coactividad,cutoff='mean',correction=False)
    print('rt =',rt)
    print('p =',p)
    #type 1 error
    Im=0
    for i in range(n_iter):
        raster_s=raster_subrogado(raster,1)
        coactividad_s=np.sum(raster_s,axis=0)
        rt_s,p_s=runstest_1samp(coactividad_s,cutoff='median',correction=False)
        if p_s<=0.05:
            Im+=1
    alphahat=Im/n_iter
    print('alphahat',alphahat)
    #type 2 error
    Im=0
    for i in range(n_iter):
        raster_s=raster_subrogado(raster,2)
        coactividad_s=np.sum(raster_s,axis=0)
        rt_s,p_s=runstest_1samp(coactividad_s,cutoff='median',correction=False)
        if p_s>0.05:
            Im+=1
    betahat=Im/n_iter
    print('betahat',betahat)
    return rt,p,alphahat,betahat

In [ ]:
def raster_subrogado(raster_real,selector):
    N,F=raster_real.shape
    raster_artificial=np.zeros((N,F))
    if selector == 1: #Mantiene número de neuronas y tiempo
        for i in range(N):
            raster_artificial[i,:]=raster_real[i,np.random.permutation(F)]
    elif selector == 2: #Mantiene ISI y tiempo
        for i in range(N):
            isi=np.diff(np.where(np.concatenate(([1],raster_real[i,:],[1]))==1)).flatten()
            t_inicio=isi[0]
            t_final=isi[-1]
            isi=np.delete(isi,[0])
            isi=np.delete(isi,len(isi)-1)
            isi_permutado=isi[np.random.permutation(len(isi))]
            t_prim_esp=np.random.randint(t_inicio+t_final)
            t_activo=np.concatenate(([0],np.cumsum(isi_permutado)))
            t_esp=t_activo+t_prim_esp-1
            raster_artificial[i,t_esp]=1
    return raster_artificial

In [ ]:
def plot_cluster_raster(raster,fps,cluster_index,name_colormap,markersize=5):
    cmap = matplotlib.cm.get_cmap(name_colormap)
    N,F=raster.shape
    actividad=np.zeros((N,1))
    orden_plot=[]
    
    plt.figure(figsize=(12,6))

    ax=plt.axes((0.05,0.35,0.75,0.6))
    count=0
    for ci in range(max(cluster_index)+1):
        indices_ci=np.where(cluster_index==ci)[0]
        for i in indices_ci:
            orden_plot.append(i)
            indices=np.where(raster[i,:]==1)[0]
            if type(cmap)==matplotlib.colors.LinearSegmentedColormap:
                plt.plot(indices,raster[i,indices]*(count+1),
                        marker='|',linestyle='None',
                        markersize=markersize,color=cmap(ci/max(cluster_index)))
            else:
                plt.plot(indices,raster[i,indices]*(count+1),
                        marker='|',linestyle='None',
                        markersize=markersize,color=cmap(ci))
            actividad[count]=np.sum(raster[i,:])*100/F
            count+=1
    ax.set_xlim(0,F-1)
    ax.set_ylim(1,N)
    plt.xticks([])
    plt.ylabel("Neuron Label")

    ax=plt.axes(((0.05,0.12,0.75,0.2)))
    coactividad=np.sum(raster,axis=0)
    fpm=fps*60
    tiempo=np.arange(0,F)/fpm

    plt.plot(tiempo,coactividad,linewidth=0.5,color='black')

    ax.set_xlim(np.min(tiempo),np.max(tiempo))
    ax.set_ylim(0,np.max(coactividad)+1)
    plt.xlabel("Time (min)")
    plt.ylabel("Coactivity")

    ax=plt.axes((0.85,0.35,0.1,0.6))
    plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
    ax.set_xlim(0,max(actividad)+1)
    ax.set_ylim(1,N)
    plt.xlabel("# Frames")
    plt.yticks([])
    
    return orden_plot

In [ ]:
def get_gephi_graph(ma,cluster_index,th):
    ma_reduced=threshold_proportional(ma, th)
    G=nx.from_numpy_matrix(ma_reduced)
    ensemble_attr_dict={}
    color_attr_dict={}
    cmap = matplotlib.cm.get_cmap('tab10')
    for i,ci in enumerate(cluster_index):
        ensemble_attr_dict[i]=str(ci)
        v=list(cmap(ci)[0:3])
        v=[vi*255 for vi in v]
        l=len(str(v))-1
        color_attr_dict[i]=str(v)[1:l]
    nx.set_node_attributes(G, ensemble_attr_dict, "ensemble")
    nx.set_node_attributes(G, color_attr_dict,"color")
    return G

In [ ]:
def get_ordered_adj_matrix(ma,cluster_index):
    ma_ordered=np.zeros(ma.shape)
    count=0
    new_order=[]
    for ci in range(max(cluster_index)+1):
        indices_ci=np.where(cluster_index==ci)[0]
        for indice in indices_ci:
            new_order.append(indice)
            count+=1
    for i in range(len(new_order)):
        indices_i=np.where(ma[new_order[i],:]>0)[0]
        for indice in indices_i:
            ma_ordered[i,np.where(new_order==indice)[0]]=ma[new_order[i],indice]
    return ma_ordered

In [ ]:
def nuevo_cluster_index(cluster_index,nuevo_orden):
    try:
        n=len(nuevo_orden)
        newCluster_index=np.zeros((len(cluster_index),))
        for i in range(max(cluster_index)+1):
            indices=np.where(cluster_index==nuevo_orden[i])[0]
            newCluster_index[indices]=int(i)
        return newCluster_index.astype(int)
    except ValueError:
        print(ValueError)
        return None

In [ ]:
def plot_raster(raster,fps,umbral=None,markersize=5):
    N,F=raster.shape
    plt.figure(figsize=(12,6))

    ax=plt.axes((0.05,0.35,0.75,0.6)) 
    for i in range(N):
        indices=np.where(raster[i,:]==1)[0]
        plt.plot(indices,raster[i,indices]*(i+1),
                marker='|',linestyle='None',
                markersize=markersize,color='black')
    ax.set_xlim(0,F-1)
    ax.set_ylim(1,N)
    plt.xticks([])
    plt.ylabel("Neuron Label")

    ax=plt.axes(((0.05,0.12,0.75,0.2)))
    coactividad=np.sum(raster,axis=0)
    fpm=fps*60
    tiempo=np.arange(0,F)/fpm
    plt.plot(tiempo,coactividad,linewidth=0.5,color='black')
    ax.set_xlim(np.min(tiempo),np.max(tiempo))
    ax.set_ylim(0,np.max(coactividad)+1)
    plt.xlabel("Time (min)")
    plt.ylabel("Coactivity")

    ax=plt.axes((0.85,0.35,0.1,0.6))
    actividad=np.sum(raster,axis=1)
    plt.plot(actividad,np.arange(1,N+1),color='black',linewidth=1)
    ax.set_xlim(0,max(actividad)+1)
    ax.set_ylim(1,N)
    plt.xlabel("# Frames")
    plt.yticks([])

In [ ]:
def get_adj_matrix(mapper):
    ma=np.zeros(mapper.graph_.shape)
    contador=0
    for i in range(ma.shape[0]):
        tam=mapper.graph_.indptr[i+1]-mapper.graph_.indptr[i]
        ma[i,mapper.graph_.indices[contador:contador+tam]]=mapper.graph_.data[contador:contador+tam]
        contador+=tam
    return ma

In [ ]:
def get_clusters_modularidad(grafo_ponderado,n_iter):
    N=len(grafo_ponderado)
    mat_mismo_grupo=np.zeros(grafo_ponderado.shape)
    for i in range(n_iter):
        cluster_index,_=community_louvain(grafo_ponderado,seed=42)
        for neu in range(N):
            module = cluster_index[neu]
            index = np.where(cluster_index==module)[0]
            mat_mismo_grupo[neu,index]+=1
    for i in range(N):
        mat_mismo_grupo[i,i]=0
    cluster_index_h,_=community_louvain(mat_mismo_grupo,seed=42)    
    cluster_index_h=cluster_index_h-1
    print("Number of Ensembles: ",str(max(cluster_index_h)+1))
    return cluster_index_h,mat_mismo_grupo

In [ ]:
def print_medias_sem(parameter):
    print('{:.3f}'.format(df[df['Condition']=='Control'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Control'][parameter].sem()))
    print('{:.3f}'.format(df[df['Condition']=='Decorticated'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Decorticated'][parameter].sem()))
    print('{:.3f}'.format(df[df['Condition']=='Parkinson'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Parkinson'][parameter].sem()))
    print('{:.3f}'.format(df[df['Condition']=='Dyskinesia'][parameter].mean()),'±','{:.3f}'.format(2*df[df['Condition']=='Dyskinesia'][parameter].sem()))

In [ ]:
def teachers_round(x):
    '''
    Do rounding such that .5 always rounds to 1, and not bankers rounding.
    This is for compatibility with matlab functions, and ease of testing.
    Authors:
        Olaf Sporns
        Mikail Rubinov
        Yusuke Adachi
        Andrea Avena
        Danielle Bassett
        Richard Betzel
        Joaquin Goni
        Alexandros Goulas
        Patric Hagmann
        Christopher Honey
        Martijn van den Heuvel
        Rolf Kotter
        Jonathan Power
        Murray Shanahan
        Andrew Zalesky
    '''
    if ((x > 0) and (x % 1 >= 0.5)) or ((x < 0) and (x % 1 > 0.5)):
        return int(np.ceil(x))
    else:
        return int(np.floor(x))

In [ ]:
def community_louvain(W, gamma=1, ci=None, B='modularity', seed=None):
    '''
    The optimal community structure is a subdivision of the network into
    nonoverlapping groups of nodes which maximizes the number of within-group
    edges and minimizes the number of between-group edges.
    This function is a fast an accurate multi-iterative generalization of the
    louvain community detection algorithm. This function subsumes and improves
    upon modularity_[louvain,finetune]_[und,dir]() and additionally allows to
    optimize other objective functions (includes built-in Potts Model i
    Hamiltonian, allows for custom objective-function matrices).
    Parameters
    ----------
    W : NxN np.array
        directed/undirected weighted/binary adjacency matrix
    gamma : float
        resolution parameter. default value=1. Values 0 <= gamma < 1 detect
        larger modules while gamma > 1 detects smaller modules.
        ignored if an objective function matrix is specified.
    ci : Nx1 np.arraylike
        initial community affiliation vector. default value=None
    B : str | NxN np.arraylike
        string describing objective function type, or provides a custom
        NxN objective-function matrix. builtin values 
            'modularity' uses Q-metric as objective function
            'potts' uses Potts model Hamiltonian.
            'negative_sym' symmetric treatment of negative weights
            'negative_asym' asymmetric treatment of negative weights
    seed : hashable, optional
        If None (default), use the np.random's global random state to generate random numbers.
        Otherwise, use a new np.random.RandomState instance seeded with the given value.
    Returns
    -------
    ci : Nx1 np.array
        final community structure
    q : float
        optimized q-statistic (modularity only)
    
    Authors:
        Olaf Sporns
        Mikail Rubinov
        Yusuke Adachi
        Andrea Avena
        Danielle Bassett
        Richard Betzel
        Joaquin Goni
        Alexandros Goulas
        Patric Hagmann
        Christopher Honey
        Martijn van den Heuvel
        Rolf Kotter
        Jonathan Power
        Murray Shanahan
        Andrew Zalesky
    '''
    rng = get_rng(seed)
    n = len(W)
    s = np.sum(W)

    #if np.min(W) < -1e-10:
    #    raise BCTParamError('adjmat must not contain negative weights')

    if ci is None:
        ci = np.arange(n) + 1
    else:
        if len(ci) != n:
            raise BCTParamError('initial ci vector size must equal N')
        _, ci = np.unique(ci, return_inverse=True)
        ci += 1
    Mb = ci.copy()
    renormalize = False
    if B in ('negative_sym', 'negative_asym'):
        renormalize = True
        W0 = W * (W > 0)
        s0 = np.sum(W0)
        B0 = W0 - gamma * np.outer(np.sum(W0, axis=1), np.sum(W0, axis=0)) / s0

        W1 = -W * (W < 0)
        s1 = np.sum(W1)
        if s1:
            B1 = W1 - gamma * np.outer(np.sum(W1, axis=1), np.sum(W1, axis=0)) / s1
        else:
            B1 = 0

    elif np.min(W) < -1e-10:
        raise BCTParamError("Input connection matrix contains negative "
            'weights but objective function dealing with negative weights '
            'was not selected')

    if B == 'potts' and np.any(np.logical_not(np.logical_or(W == 0, W == 1))):
        raise BCTParamError('Potts hamiltonian requires binary input matrix')

    if B == 'modularity':
        B = W - gamma * np.outer(np.sum(W, axis=1), np.sum(W, axis=0)) / s
    elif B == 'potts':
        B = W - gamma * np.logical_not(W)
    elif B == 'negative_sym':
        B = (B0 / (s0 + s1)) - (B1 / (s0 + s1))
    elif B == 'negative_asym':
        B = (B0 / s0) - (B1 / (s0 + s1))
    else:
        try:
            B = np.array(B)
        except:
            raise BCTParamError('unknown objective function type')

        if B.shape != W.shape:
            raise BCTParamError('objective function matrix does not match '
                                'size of adjacency matrix')
        if not np.allclose(B, B.T):
            print ('Warning: objective function matrix not symmetric, '
                   'symmetrizing')
            B = (B + B.T) / 2
    
    Hnm = np.zeros((n, n))
    for m in range(1, n + 1):
        Hnm[:, m - 1] = np.sum(B[:, ci == m], axis=1)  # node to module degree
    H = np.sum(Hnm, axis=1)  # node degree
    Hm = np.sum(Hnm, axis=0)  # module degree

    q0 = -np.inf
    # compute modularity
    q = np.sum(B[np.tile(ci, (n, 1)) == np.tile(ci, (n, 1)).T]) / s

    first_iteration = True

    while q - q0 > 1e-10:
        it = 0
        flag = True
        while flag:
            it += 1
            if it > 1000:
                raise BCTParamError('Modularity infinite loop style G. '
                                    'Please contact the developer.')
            flag = False
            for u in rng.permutation(n):
                ma = Mb[u] - 1
                dQ = Hnm[u, :] - Hnm[u, ma] + B[u, u]  # algorithm condition
                dQ[ma] = 0

                max_dq = np.max(dQ)
                if max_dq > 1e-10:
                    flag = True
                    mb = np.argmax(dQ)

                    Hnm[:, mb] += B[:, u]
                    Hnm[:, ma] -= B[:, u]  # change node-to-module strengths

                    Hm[mb] += H[u]
                    Hm[ma] -= H[u]  # change module strengths

                    Mb[u] = mb + 1

        _, Mb = np.unique(Mb, return_inverse=True)
        Mb += 1

        M0 = ci.copy()
        if first_iteration:
            ci = Mb.copy()
            first_iteration = False
        else:
            for u in range(1, n + 1):
                ci[M0 == u] = Mb[u - 1]  # assign new modules

        n = np.max(Mb)
        b1 = np.zeros((n, n))
        for i in range(1, n + 1):
            for j in range(i, n + 1):
                # pool weights of nodes in same module
                bm = np.sum(B[np.ix_(Mb == i, Mb == j)])
                b1[i - 1, j - 1] = bm
                b1[j - 1, i - 1] = bm
        B = b1.copy()

        Mb = np.arange(1, n + 1)
        Hnm = B.copy()
        H = np.sum(B, axis=0)
        Hm = H.copy()

        q0 = q

        q = np.trace(B)  # compute modularity
    
    # Workaround to normalize
    if not renormalize:
        return ci, q/s
    else:
        return ci, q

In [ ]:
def threshold_proportional(W, p, copy=True):
    '''
    This function "thresholds" the connectivity matrix by preserving a
    proportion p (0<p<1) of the strongest weights. All other weights, and
    all weights on the main diagonal (self-self connections) are set to 0.
    If copy is not set, this function will *modify W in place.*
    Parameters
    ----------
    W : np.ndarray
        weighted connectivity matrix
    p : float
        proportional weight threshold (0<p<1)
    copy : bool
        if True, returns a copy of the matrix. Otherwise, modifies the matrix
        in place. Default value=True.
    Returns
    -------
    W : np.ndarray
        thresholded connectivity matrix
    Notes
    -----
    The proportion of elements set to 0 is a fraction of all elements
    in the matrix, whether or not they are already 0. That is, this function
    has the following behavior:
    >> x = np.random.random_sample((10,10))
    >> x_25 = threshold_proportional(x, .25)
    >> np.size(np.where(x_25)) #note this double counts each nonzero element
    46
    >> x_125 = threshold_proportional(x, .125)
    >> np.size(np.where(x_125))
    22
    >> x_test = threshold_proportional(x_25, .5)
    >> np.size(np.where(x_test))
    46
    That is, the 50% thresholding of x_25 does nothing because >=50% of the
    elements in x_25 are aleady <=0. This behavior is the same as in BCT. Be
    careful with matrices that are both signed and sparse.
    Authors:
        Olaf Sporns
        Mikail Rubinov
        Yusuke Adachi
        Andrea Avena
        Danielle Bassett
        Richard Betzel
        Joaquin Goni
        Alexandros Goulas
        Patric Hagmann
        Christopher Honey
        Martijn van den Heuvel
        Rolf Kotter
        Jonathan Power
        Murray Shanahan
        Andrew Zalesky
    '''
    if copy:
        W = W.copy()
    n = len(W) # number of nodes
    np.fill_diagonal(W, 0) # clear diagonal

    if np.allclose(W, W.T): # if symmetric matrix
        W[np.tril_indices(n)] = 0 # ensure symmetry is preserved
        ud = 2 # halve number of removed links
    else:
        ud = 1

    ind = np.where(W) # find all links

    I = np.argsort(W[ind])[::-1] # sort indices by magnitude

    en = int(teachers_round((n * n - n) * p / ud)) # number of links to be preserved

    W[(ind[0][I][en:], ind[1][I][en:])] = 0  # apply threshold
    #W[np.ix_(ind[0][I][en:], ind[1][I][en:])]=0

    if ud == 2: # if symmetric matrix
        W[:, :] = W + W.T # reconstruct symmetry

    return W